In [9]:
class Elevator():
    """
    Defines the Elevator class.
    
    Class Instance Atrributes:
        current_floor (int): The floor the elevator is on.
        floor_range (range): The floors the elevator can move to.
        requests_by_floor (dict): Contains the floor requests are on as keys, and lists of passengers that made the requests as values.
        pending_requests (list): List of people waiting for elevator (their indexes are treated as the order in which the buttons were pressed).
        passengers_in_elevator (list): List of passengers in the elevator.
        capacity (int): Maximum number of passengers allowed in elevator at once.
    """
    
    def __init__(self, floor_range, current_floor, capacity):
        """The constructor for the Elevator class"""
        
        self.current_floor = current_floor
        self.floor_range = floor_range
        self.requests_by_floor = {}
        self.pending_requests = []
        self.passengers_in_elevator = []
        self.capacity = capacity

    def move_up(self):
        """Moves the elevator up one floor."""
        
        if self.current_floor < self.floor_range[-1]:
            self.current_floor += 1
        else:
            raise ValueError("Can't keep moving up.")


    def move_down(self):
        """Moves the elevator down one floor."""
        
        if self.current_floor > self.floor_range[0]:
            self.current_floor -= 1
        else:
            raise ValueError("Can't keep moving down.")

    def request(self,on_floor,passenger):
        """
        Tells the elevator that someone pushed the call button.
        
        Also, on which floor the button was pushed, and adds the next passenger to the requests_by_floor list.
        
        Parameters:
            passenger(obj): The passenger that pushed the elevator button
        """
        
        self.pending_requests.append(passenger)

        if self.requests_by_floor.has_key(on_floor):
            self.requests_by_floor[on_floor].append(passenger)
        else:
            self.requests_by_floor[on_floor] = [passenger]

In [10]:
import random

class Passenger:
    """
    Defines the Passenger class.
 
    Class Instance Attributes:
        self.starting_floor (int): Floor that the passenger is taking the elevator from.
        self.destination_floor (int): The destination floor of the passenger.
    """
    def __init__(self, floor_range):
        """
        The constructor for Passenger class

        Parameters:
            floor_range (list): Number of floors in the building.
        """ 
        self.starting_floor = random.choice(floor_range)
        while True:
            self.destination_floor = random.choice(floor_range)
            if self.destination_floor != self.starting_floor:
                break

In [11]:
class Building():
    """
    Defines the Building class
    
    Class Instance Attributes:
        floor_range (list): Number of floors in the building.
        elevator (Elevator instance): The elevator in the building.
        passengers (list of Passenger instances): List of all of the passengers that will be taking the elevator.
    """  
    
    def __init__(self,no_of_floors=13,no_of_passengers=50,capacity=10):
        """
        The constructor for the Building class.
        
        Parameters:
            no_of_floors (int): Number of floors in the building
            no_of_passengers (int): Number of passengers in the building
            capacity (int): Capacity of the elevator
        """
        
        self.floor_range = range(no_of_floors)
        self.elevator = Elevator(self.floor_range,0,capacity)
        self.passengers = []
        for x in range(no_of_passengers):
            self.add_passenger()

    def add_passenger(self):
        """
        This function adds a passenger to the building.
        """
        
        new_passenger = Passenger(self.floor_range)
        self.passengers.append(new_passenger)
        self.elevator.request(new_passenger.starting_floor,new_passenger)

In [12]:
import math


class Simulation():
    """
    Simulates movement of the elevator, runs the step counter, simulates passengers moving in and out of the elevator.

    Class Attributes:
        MOVE_UP
        STOP_AND_OPEN
        MOVE_DOWN

    Class Instance Attributes:
        extra_steps_taken_per_passenger (list): List of integers recording the extra steps each passenger took.
        building (Building instance): Creates an instance of the building class.
    """

    MOVE_UP = 1
    STOP_AND_OPEN = 0
    MOVE_DOWN = -1

    def __init__(self, no_of_floors, no_of_passengers, capacity, step_function, print_intermediate_states=False):
        """
        The constructor for the Simulation class.

        Parameters:
            no_of_floors (int): Number of floors in the building
            no_of_passengers (int): Number of passengers in the building
            capacity (int): Capacity of the elevator
            step_function (function): Calculates the number of the steps taken
            print_intermediate_states (binary): Sets a default to not print the state of the elevator at each time-step.
        """

        self.step_count = -1
        self.extra_steps_taken_per_passenger = []
        self.building = Building(no_of_floors,no_of_passengers,capacity)
        state = {}
        elevator = self.building.elevator
        actions = {          # Creates a dictionary that makes the actions of the elevator easier to read.
            Simulation.MOVE_UP: elevator.move_up,
            Simulation.STOP_AND_OPEN: self.stop_and_open_elevator,
            Simulation.MOVE_DOWN: elevator.move_down,
        }

        #This loop runs the whole simulation. Stops when elevator is empty and there are no more requests.
        while len(elevator.pending_requests) > 0 or len(elevator.passengers_in_elevator) > 0:
            self.step_count += 1
            action = step_function(elevator, state)
            actions[action]()
#             if print_intermediate_states:
#                   self.print_state()

        extra_steps = self.extra_steps_taken_per_passenger
        percentile =len(extra_steps) * (.95)
        self.steps_taken_per_passenger_average = float(sum(extra_steps)) / len(extra_steps)
        self.steps_taken_per_passenger_95_percentile = (
            (extra_steps[int(math.ceil(percentile))] + extra_steps[int(math.floor(percentile))])  / 2
        ) #Takes an average of the two values closer to the float that should be 95th percentile
        self.print_end_state()

    def stop_and_open_elevator(self):
        """The function stops the elevator and delivers passengers at the requested floors."""

        elevator = self.building.elevator

        for passenger in elevator.passengers_in_elevator:
            if passenger.destination_floor == elevator.current_floor:
                steps_taken = self.step_count
                minimum_steps_needed = abs(passenger.destination_floor - passenger.starting_floor) + 1
                extra_steps_taken = steps_taken - minimum_steps_needed
                self.extra_steps_taken_per_passenger.append(extra_steps_taken)
                elevator.passengers_in_elevator.remove(passenger)
#                 print("Passenger reached destination")

        if elevator.requests_by_floor.has_key(elevator.current_floor):
            for passenger in elevator.requests_by_floor[elevator.current_floor]:
                if len(elevator.passengers_in_elevator) == elevator.capacity:
                    break

#                 print("Passenger entered elevator")
                elevator.passengers_in_elevator.append(passenger)
                elevator.pending_requests.remove(passenger)
                elevator.requests_by_floor[elevator.current_floor].remove(passenger)

    def print_state(self):
        """Prints the current state of the simulation."""

        elevator = self.building.elevator
        waiting_count = len(elevator.pending_requests)
        in_elevator_count = len(elevator.passengers_in_elevator)
        done_count = len(self.building.passengers) - waiting_count - in_elevator_count
        print("Step count of steps is ", self.step_count)
        print("waiting vs elevator vs done", waiting_count, in_elevator_count, done_count)
        print("Current floor", elevator.current_floor)


    def print_end_state(self):  
        """Prints the relevant outcomes of the simulation."""

        print("Average number of extra steps taken is ", self.steps_taken_per_passenger_average)
        print("The 95th percentile of extra steps taken is ", self.steps_taken_per_passenger_95_percentile)
        print("Total number of steps is ", self.step_count)

In [13]:
def example_strategy_step_function(elevator, state):
    """
    The function implements the strategy of an elevator starting at the ground floor and moving to the last floor, stoping at each floor.
    
    Parameters:
        elevator (Elevator instance): 
        state (dict): The dictionary that is keeping the track of the state, in this case the direction in which elevator is moving
    """
    
    MOVING_UP = "moving_up"
    STOPPED_AND_OPENED = "stopped_and_opened"
    if not state:
        state[MOVING_UP] = True
        state[STOPPED_AND_OPENED] = False

    state[STOPPED_AND_OPENED] = not state[STOPPED_AND_OPENED]

    if state[STOPPED_AND_OPENED]:
        return Simulation.STOP_AND_OPEN
    else:
        current_floor = elevator.current_floor
        top_floor = elevator.floor_range[-1]
        bottom_floor = elevator.floor_range[0]

        if state[MOVING_UP] == True and current_floor == top_floor:
            state[MOVING_UP] = False
        elif state[MOVING_UP] == False and current_floor == bottom_floor:
            state[MOVING_UP] = True

        if state[MOVING_UP] == True:
            return Simulation.MOVE_UP
        else:
            return Simulation.MOVE_DOWN

modeling = Simulation(no_of_floors=100,no_of_passengers=1000, capacity=15, step_function=example_strategy_step_function)

('Average number of extra steps taken is ', 4545.048)
('The 95th percentile of extra steps taken is ', 8820)
('Total number of steps is ', 9990)


In [14]:
def first_come_step_function(elevator,state):
    """
    Defines the "First Come, First Serve" elevator strategy.

    Parameters:
        elevator (Elevator object):
        state (dict): The dictionary that is keeping the track of the state, in this case the direction in which elevator is moving.
    """
    if not elevator.passengers_in_elevator:
        if elevator.current_floor < elevator.pending_requests[0].starting_floor:
            return Simulation.MOVE_UP
        elif elevator.current_floor > elevator.pending_requests[0].starting_floor:
            return Simulation.MOVE_DOWN
        else:
            return Simulation.STOP_AND_OPEN
    else:
        if elevator.current_floor < elevator.passengers_in_elevator[0].destination_floor:
            return Simulation.MOVE_UP
        elif elevator.current_floor > elevator.passengers_in_elevator[0].destination_floor:
            return Simulation.MOVE_DOWN
        else:
            return Simulation.STOP_AND_OPEN

modeling = Simulation(no_of_floors=100,no_of_passengers=1000, capacity=15, step_function=first_come_step_function)


('Average number of extra steps taken is ', 15339.618)
('The 95th percentile of extra steps taken is ', 29367)
('Total number of steps is ', 32307)


In [15]:
from collections import Counter


def democratic_strategy_step_function(elevator, state):
    """Defines the Democratic Strategy, picking up passengers who are on floors the elevator is passing, and dropping off passengers at the floor most have as their destination.

    Parameters:
        elevator (Elevator object):
        state (dict): The dictionary that is keeping the track of the state, in this case the direction in which elevator is moving.
    """

    for passenger in elevator.passengers_in_elevator:  # Checks if we are on the destination of someone.
        if passenger.destination_floor == elevator.current_floor:
            return Simulation.STOP_AND_OPEN

    if (len(elevator.passengers_in_elevator) < elevator.capacity  # Checks if we can pick up people on
                                                                  # current floor.
       and elevator.current_floor in elevator.requests_by_floor
       and elevator.requests_by_floor[elevator.current_floor]):
        return Simulation.STOP_AND_OPEN

    floors = None
    if elevator.passengers_in_elevator:  # This block of code decides how to moves the elevator.
        floors = [ passenger.destination_floor for passenger in elevator.passengers_in_elevator ]
    else:
        floors = [ passenger.starting_floor for passenger in elevator.pending_requests ]
    mode_floor = Counter(floors).most_common(1)[0][0]
    return [ Simulation.MOVE_UP, Simulation.MOVE_DOWN, ][mode_floor < elevator.current_floor]

modeling = Simulation(no_of_floors=100,no_of_passengers=1000, capacity=15, step_function=democratic_strategy_step_function)

('Average number of extra steps taken is ', 3883.51)
('The 95th percentile of extra steps taken is ', 7640)
('Total number of steps is ', 8854)
